In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
#import gym
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from itertools import combinations
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from scipy.special import iv  # Modified Bessel function of the first kind
from scipy.stats import rice
%matplotlib inline

# 1. Problem Formulation 
UAV should serve the following roles and purposes:
1. Base Station
2. Relay
3. Signal Jammer

Objective for optimisation:
Minimum average secrecy rate
Achieve this by optimising:
1. UAV Trajectory
2. UAV Transmit Power Allocation
3. UAV Energy Efficiency

Subject to the following constraints:
1. UAV Energy Consumption
2. UAV Mission Time (secrecy & energy)
3. Environmental Conditions (weather, LoS, infrastructure, etc.)
4. UAV Altitude & Range
5. Rician Channel Fading (kind of factors into communications range, signal power, etc.)
6. Shannon Limit/Data Rates

## Project Roadmap
### i. Classical Implementation
1. Implement the optimisation problem using frameworks like Pyomo or just "manual" calculations as well as using ML/DRL for comparison with the LQ-DRL approach
2. Test the optimisation algorithm with different scenarios (urban vs clear/rural environment and with different weather conditions if possible)
3. Determine which parts of the algorithm ought to be solved with the use of quantum computing (main focus should be the LQ-DRL approach as per Anshu's recommendation)
4. Implement the quantum/classical hybrid algorithm and compare the results with the classical method in terms of different metrics 
5. Compare different embedding techniques and results
6. Try to ameliorate/append the LQ-DRL algorithm from Silvirianti et. al (2025) -- vague for now, clarify as you go on

## i. Objective Function


$
R_{sec} = 
$

# Mathematical Workings
## Energy Consumption Model
Adapted from Silvirianti et. al (2025)
\
$
\Large
E_{cons}(t) = \sum_{n=1}^{N} \sum_{k=1}^{K_n} \left (\frac{\sum_{i=1}^{2} n_{i} g c(t)}{\mathbf{k}\textit{z}} + 
\frac{((\sum_{i=1}^{2}n_i)g)^{\frac{3}{2}}}{\sqrt{2 z \zeta \theta}} +
\Lambda \frac{c(t)}{v(t)} + 
P_{k, n}^{Tx}(t) R_{k, n}(t) \right)
$
\
Where
$
n_i = 
$
UAV framework & battery
\
$
g = 
$
Acceleration due to gravity ($9.81 m s^{-2}$)
\
$
c(t) = \sqrt{\sum_{i=t-1}^{t} (x_i - x_{i-1})^{2} + (y_i - y_{i-1})^{2} + (H_i - H_{i-1})^{2}}
$
\
i.e., $c(t)$ = The location of a UAV at time $t$
\
$
\mathbf{k} = 
$
Lift-to-Drag ratio
\
$
\textit{z} = 
$
Number of UAV Rotors
\
$
\zeta = 
$
Air Density
\
$
\theta = 
$
Spinning Blade of One Rotor
\
$
\Lambda = 
$
Avionics Power for UAV
\
$
v(t) = 
$
Velocity of UAV at time $t$
\
$
P_{k, n}^{Tx}(t) = 
$
Communication power of the $k^{th}$ user
\
$
R_{k, n}(t) = 
$
Achievable Sum Rate of $k^{th}$ user in $n^{th}$ group

## Energy Efficiency
$
\Large
\eta(T) = \int_{t=1}^{T} \frac{\sum_{n=1}^{N} \sum_{k=1}^{K_n} R_{k, n}(t)}{E_{cons}(t)} dt
$
\
Maximise $\eta(T)$ subject to
\
Allocated NOMA Group Power Constraint:
\
$
\sum_{n=1}^{N} \sum_{k=1}^{K_n} P_{k, n}^{Tx}(t) \le P_{UAV}^{Tx}
$
\
Power Coefficient Constraints
\
$
0 \lt \sum_{k=1}^{K_n} \delta_{k, n} \le 1
$
\
$
\delta_{1, n} \lt \dots \lt \delta_{K_n, n}
$
\
Rate of Transmission Constraint:
\
$
R_{k, n}(t) \ge R_{min}
$
\
UAV Range & Altitude Constraints:
\
$
x_{min} \le x(t) \le x_{max}
$
\
$
y_{min} \le y(t) \le y_{max}
$
\
$
H_{min} \le H(t) \le H_{max}
$

## Communication Channels
### Rician Channel Model
$
\Large
s(t) = cos(\omega_{c}t)
$
\
$
\Large
v(t) = C \cdot cos(\omega_{c}t) + \sum_{n=1}^{N} r_n cos(\omega_{c}t + f_n)
$
\
$
\Large
K = \frac{v^2}{2\sigma^2}
$
\
Where $v^2$ is the signal power in the direct path and $2\sigma^2$ is the signal power of the scattered paths, thus, $K$ is the ratio between these 2 quantities.
\
Statistically, this can be viewed as the most dominant LoS path in the channel (centre of a Bell curve distribution, i.e., the mean $\mu$) and the standard deviations either side of it ($\sigma$ in a statistical distribution). 
\
$
\Large
\Omega = v^2 + 2\sigma^2
$
\
$\Omega$ can be seen to be the total power from both paths. 
\
$
\Large
v^2 = \frac{K}{1+K}\Omega
$
\
$
\Large
2\sigma^2 = \frac{\Omega}{1+K}
$
\
The resulting PDF:
\
$
\Large
f(x) = \frac{2(K+1)x}{\Omega}exp\left(-K-\frac{(K+1)x^2}{\Omega}\right) I_0 \left (2 \sqrt{\frac{K(K+1)}{\Omega}}x \right)
$

## Deep Reinforcement Learning
Policy $\pi(s, a)$ is determined by the Agent.
\
Value of the policy is determined by the expectation value of the policy $\pi$ based on if the state starts in a particular state $s_0$ with a discount rate $\gamma$ at time $t$.  
\
$
V_{\pi}(s) = E(\sum_{t} \gamma^t r_t | s_0 = s) 
$
\
Problem is an optimisation problem to solve for $\pi$.
\
Markov Decision Process (MDP) is a probabilistic process for going from state $s_t$ to the next state $s_{t+1}$. 

In [32]:
%load_ext autoreload
%autoreload 2
from uav_secrecy_env import UAVSecrecyEnv

env = UAVSecrecyEnv()
env = Monitor(env)
env = DummyVecEnv([lambda: env])

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50000)
model.save("ppo_uav_secrecy")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -1.19    |
| time/              |          |
|    fps             | 1537     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -1.42       |
| time/                   |             |
|    fps                  | 1087        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006795961 |
|    clip_fraction        | 0.0924      |
|    clip_range           | 0.2         |
|    entr

In [33]:
%load_ext autoreload
%autoreload 2
from uav_secrecy_env import UAVSecrecyEnv

env = UAVSecrecyEnv()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50000)
model.save("ppo_uav_secrecy")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -2.6     |
| time/              |          |
|    fps             | 1500     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -1.8        |
| time/                   |             |
|    fps                  | 1114        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008134555 |
|    clip_fracti